In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import datetime as dt

In [3]:
URL = "https://fiis.com.br/rendimentos/?ticker="
dr = webdriver.Chrome()
dr.implicitly_wait(3)

In [4]:
dr.get(URL)

In [5]:
soup = BeautifulSoup(dr.page_source, "lxml")
dr.quit()

In [6]:
FII_list = soup.findAll("div",attrs={"class":"accordion-body"})

In [7]:
missing_FIIs = []

In [8]:

new_FII_list = []

for FII in FII_list:
    table = FII.findAll("div", attrs={"class":"yieldChart__table__bloco yieldChart__table__bloco--rendimento"})
    for row in table:
        FII_name = FII['data-accordion-body']
        temp_dict = {"Nome" : FII_name}
        
        FII_row_data = row.findAll("div", attrs={"class":"table__linha"})
        
        temp_dict["Data"] = FII_row_data[1].text.strip()
        temp_dict["Pagamento"] = FII_row_data[2].text.strip()
        temp_dict["Cotação"] = FII_row_data[3].text.strip()
        temp_dict["Yield"] = FII_row_data[4].text.strip()
        temp_dict["Rendimento"] = FII_row_data[5].text.strip()

        new_FII_list.append(temp_dict)
        
        #temp_df = pd.DataFrame([FII_row_data],columns = ["Nome","Tipo", "Data", "Pagamento", "Cotação", "Yield", "Rendimento"])
        #output = pd.concat([output,temp_df], ignore_index=True)

In [9]:
output = pd.DataFrame.from_dict(new_FII_list)

In [10]:
output

,Nome,Data,Pagamento,Cotação,Yield,Rendimento
0,AAGR11,08.10.2024,15.10.2024,"R$ 96,92","1,11%","R$ 1,07"
1,AAGR11,06.09.2024,13.09.2024,"R$ 95,80","1,17%","R$ 1,12"
2,AAGR11,08.08.2024,15.08.2024,"R$ 96,57","1,21%","R$ 1,17"
3,AAGR11,08.07.2024,15.07.2024,"R$ 95,37","1,12%","R$ 1,06"
4,AAGR11,07.06.2024,14.06.2024,"R$ 93,53","1,20%","R$ 1,13"
...,...,...,...,...,...,...
22712,ZAVI11,29.12.2022,13.01.2023,"R$ 124,76","0,98%","R$ 1,22"
22713,ZAVI11,30.11.2022,14.12.2022,"R$ 124,76","0,95%","R$ 1,18"
22714,ZAVI11,31.10.2022,16.11.2022,"R$ 0,00","0,00%","R$ 1,14"
22715,ZIFI11,30.06.2022,14.07.2022,"R$ 0,00","0,00%","R$ 3,31"


In [11]:
output.to_csv("raw.csv")

In [7]:
output = pd.read_csv("raw.csv", index_col=0)

0    R$ 1,07
1    R$ 1,12
2    R$ 1,17
3    R$ 1,06
4    R$ 1,13
5    R$ 1,19
6    R$ 1,05
7    R$ 0,89
Name: Rendimento, dtype: object

In [13]:
fixed_format = {  '\\%' : '',
                'R\\$' : '',
                '\\.' : '',
                ',' : '.',
                ' ' : ''}



output = output.replace({"Cotação" : fixed_format, 
                "Yield": fixed_format,
                "Rendimento" : fixed_format}, 
                regex =True)




In [14]:
output["Cotação"] = output["Cotação"].apply(pd.to_numeric)
output["Yield"] = output["Yield"].apply(pd.to_numeric)
output["Rendimento"] = output["Rendimento"].apply(pd.to_numeric)

output["Data"] = pd.to_datetime(output["Data"],format="%d.%m.%Y")
output["Pagamento"] = pd.to_datetime(output["Pagamento"],format="%d.%m.%Y",errors='coerce')

In [20]:
df = pd.read_csv("treated.csv", index_col=0)
df[df["Nome"]=="AAGR11"]["Rendimento"].tolist()

[1.07, 1.12, 1.17, 1.06, 1.13, 1.19, 1.05, 0.89]

In [34]:
output.sort_values(by='Data', ascending = False)

,Nome,Data,Pagamento,Cotação,Yield,Rendimento
4627,CXRI11,2024-11-01,2024-11-12,73.48,0.80,0.59
15795,RBIF11,2024-11-01,2024-11-25,85.50,0.21,0.18
14084,OPTM11,2024-11-01,2024-11-14,0.00,0.00,0.14
4195,CRFF11,2024-11-01,2024-11-12,73.00,0.84,0.61
7735,GGRC11,2024-11-01,2024-11-08,10.28,0.97,0.10
...,...,...,...,...,...,...
18134,SAIC11B,2016-06-01,2016-06-15,0.00,0.00,0.73
4728,CXRI11,2016-06-01,2016-06-10,0.00,0.00,0.99
11863,KNRI11,2016-01-31,2016-02-14,0.00,0.00,0.92
2650,BRCR11,2006-05-06,2016-05-16,0.00,0.00,5.15


In [53]:
output[output['Data'].dt.year == 2024]

,Nome,Data,Pagamento,Cotação,Yield,Rendimento
0,AAGR11,2024-10-08,2024-10-15,96.92,1.11,1.07
1,AAGR11,2024-09-06,2024-09-13,95.80,1.17,1.12
2,AAGR11,2024-08-08,2024-08-15,96.57,1.21,1.17
3,AAGR11,2024-07-08,2024-07-15,95.37,1.12,1.06
4,AAGR11,2024-06-07,2024-06-14,93.53,1.20,1.13
...,...,...,...,...,...,...
22695,ZAVI11,2024-05-31,2024-06-13,107.48,1.00,1.07
22696,ZAVI11,2024-04-30,2024-05-15,121.87,0.97,1.18
22697,ZAVI11,2024-03-28,2024-04-11,127.36,1.05,1.34
22698,ZAVI11,2024-02-29,2024-03-13,128.02,1.05,1.34


In [54]:
output[output['Data'].dt.year == 2024].groupby("Nome").mean().sort_values("Yield", ascending=False)

,Data,Pagamento,Cotação,Yield,Rendimento
Nome,,,,,
HDEL11,2024-06-28 00:00:00,2024-07-12 00:00:00,110.000000,9.180000,10.100000
LTMT11,2024-10-07 00:00:00,2024-10-21 00:00:00,95.190000,8.800000,8.380000
IDFI11,2024-05-30 08:00:00,2024-06-06 13:20:00,2.582222,5.108889,0.123333
HGPO11,2024-06-14 16:48:00,2024-06-29 02:24:00,297.952000,4.737000,14.495000
BBFI11B,2024-03-30 09:36:00,2024-04-14 19:12:00,1248.376000,4.330000,56.082000
...,...,...,...,...,...
VIDS11,2024-06-22 12:00:00,2024-07-01 12:00:00,0.000000,0.000000,22.420000
VGII11,2024-06-14 16:48:00,2024-06-21 21:36:00,0.000000,0.000000,1.121000
BBIM11,2024-05-15 00:00:00,2024-05-29 12:00:00,0.000000,0.000000,0.087500


In [ ]:
output.to_csv("treated.csv")

